In [17]:
import sys
sys.path.append('../../configs')
sys.path.append('../../utils')
sys.path.append('../../tfops')

# ../../utils
from datasetmanager import DATASETMANAGER_DICT
from format_op import params2id, listformat1, listformat2, numformat2
from csv_op import CsvWriter2, CsvWriter, read_csv
from writer import create_muldir, write_pkl
from reader import read_pkl

from transform_op import apply_tf_op, pairwise_distance_euclid_efficient
# ./
from local_config import ACTIVATE_K_SET, K_SET,\
                         DATASET, GPU_ID, BATCH_SIZE, NSCLASS, EPOCH,\
                         CONV_NAME, EMBED_M, LOSS_TYPE,\
                         BUCKET_D, ACTIVATE_K,\
                         HASH_DECAY_TYPE, HASH_DECAY_PARAM_TYPE,\
                         HASH_LOSS_TYPE, HASH_DISTANCE_TYPE,\
                         HASH_LAMBDA, HASH_MARGIN_ALPHA,\
                         PAIRWISE_LAMBDA,\
                         COLORS, RESULT_DIR, RESULT_DIR_READ
from deepmetric import DeepMetric


import numpy as np
import itertools

In [2]:
class struct:
    pass

In [3]:
args = struct()

In [7]:
args.gpu=GPU_ID
args.dataset=DATASET
args.nbatch =BATCH_SIZE
args.nsclass = NSCLASS
args.epoch =EPOCH
args.conv = CONV_NAME
args.ltype = LOSS_TYPE
args.m = EMBED_M
args.hdt = HASH_DISTANCE_TYPE
args.hltype = HASH_LOSS_TYPE
args.hdtype = HASH_DECAY_TYPE
args.hdptype = HASH_DECAY_PARAM_TYPE
args.hma = HASH_MARGIN_ALPHA
args.hlamb = HASH_LAMBDA
args.plamb = PAIRWISE_LAMBDA
args.k = ACTIVATE_K
args.d = BUCKET_D

In [8]:
args.k=1
args.nbatch=128
args.nsclass=64
args.d = 32
args.plamb=0.1
args.hdptype='a3'

In [9]:
args.ltype=args.hltype
args.m = args.d
if args.ltype=='npair':
    args.nsclass = args.nbatch//2 

FILE_ID = params2id(args.dataset, args.conv, args.ltype, args.m)

if args.hltype=='npair':
    args.nsclass = args.nbatch//2
    assert args.nsclass==args.nbatch//2, "In npair loss, nsclass should be half of batch size"

if args.hltype=='npair':
    HASH_FILE_ID = FILE_ID+'_'+params2id(args.nbatch, args.nsclass, args.hltype, args.hdt, args.d, args.k, args.hlamb, args.plamb, args.hdtype, args.hdptype)
else:
    HASH_FILE_ID = FILE_ID+'_'+params2id(args.nbatch, args.nsclass, args.hltype, args.hdt, args.d, args.k, args.hma, args.plamb, args.hdtype, args.hdptype)

In [10]:
HASH_FILE_ID

'imgnet32_conv2_npair_32_128_64_npair_hash1_32_1_0.003_0.1_stair_a3'

In [11]:
args.gpu=1

In [12]:
SAVE_DIR = RESULT_DIR_READ+'metric/save/%s/'%FILE_ID
HASH_LOG_DIR = RESULT_DIR+'hash/log/'
HASH_CSV_DIR = RESULT_DIR+'hash/csv/'
HASH_PKL_DIR = RESULT_DIR+'hash/pkl/'
HASH_ASSET_DIR = RESULT_DIR+'hash/asset/'
HASH_SAVE_DIR = RESULT_DIR+'hash/save/%s/'%HASH_FILE_ID
HASH_BOARD_DIR = RESULT_DIR+'hash/board/%s/'%HASH_FILE_ID

In [13]:
datasetmanager = DATASETMANAGER_DICT[args.dataset]
# load data
dm_train, dm_val, dm_test = datasetmanager(args.hltype, nsclass=args.nsclass) if args.hltype in ['triplet', 'npair'] else datasetmanager(args.hltype)
dm_train.print_shape()
dm_val.print_shape()
dm_test.print_shape()

Npy is read from /data_large/unsynced_store/maestro/dataset/Imagenet32_processed/train_img.npy
Npy is read from /data_large/unsynced_store/maestro/dataset/Imagenet32_processed/train_label.npy
Npy is read from /data_large/unsynced_store/maestro/dataset/Imagenet32_processed/val_img.npy
Npy is read from /data_large/unsynced_store/maestro/dataset/Imagenet32_processed/val_label.npy
Npy is read from /data_large/unsynced_store/maestro/dataset/Imagenet32_processed/test_img.npy
Npy is read from /data_large/unsynced_store/maestro/dataset/Imagenet32_processed/test_label.npy
Image shape : (1153044, 32, 32, 3)
Label shape : (1153044,)
Image shape : (50000, 32, 32, 3)
Label shape : (50000,)
Image shape : (50000, 32, 32, 3)
Label shape : (50000,)


In [14]:
model = DeepMetric(dm_train, dm_val, dm_test, HASH_LOG_DIR+HASH_FILE_ID+'.log', args)
model.build(False)
model.restore(save_dir=SAVE_DIR)
model.prepare_test()
model.build_hash()
model.set_up_train_hash()
model.restore_hash(save_dir=HASH_SAVE_DIR)

[0204 06:48:36] Setting train_dataset starts
[0204 06:48:36] Setting train_dataset ends
[0204 06:48:36] Setting val_dataset starts
[0204 06:48:36] Setting val_dataset ends
[0204 06:48:36] Setting test_dataset starts
[0204 06:48:36] Setting test_dataset ends
[0204 06:48:36] Model building starts
[0204 06:48:40] Model building ends
[0204 06:48:40] Restoring model starts...
[0204 06:48:40] Restoring from ../../../exp_results/imagnet_exps/hash_metric1/metric/save/imgnet32_conv2_npair_32/model-100


Variables loaded from pretrained network
    
INFO:tensorflow:Restoring parameters from ../../../exp_results/imagnet_exps/hash_metric1/metric/save/imgnet32_conv2_npair_32/model-100


[0204 06:48:40] Restoring parameters from ../../../exp_results/imagnet_exps/hash_metric1/metric/save/imgnet32_conv2_npair_32/model-100
[0204 06:48:40] Restoring model done.
[0204 06:48:40] Model preparing test
batch: 100%|##########| 391/391 [00:08<00:00, 45.16it/s]
[0204 06:49:07] Calculating pairwise distance of validation data
[0204 06:52:29] Model building train hash starts
[0204 06:52:29] Model building train hash ends
[0204 06:52:29] Model setting up train hash starts
[0204 06:52:30] Model setting up train hash ends
[0204 06:52:30] Restoring model starts...
[0204 06:52:30] Restoring from ../../../exp_results/imagnet_exps/hash_metric2/hash/save/imgnet32_conv2_npair_32_128_64_npair_hash1_32_1_0.003_0.1_stair_a3/model-12


INFO:tensorflow:Restoring parameters from ../../../exp_results/imagnet_exps/hash_metric2/hash/save/imgnet32_conv2_npair_32_128_64_npair_hash1_32_1_0.003_0.1_stair_a3/model-12


[0204 06:52:30] Restoring parameters from ../../../exp_results/imagnet_exps/hash_metric2/hash/save/imgnet32_conv2_npair_32_128_64_npair_hash1_32_1_0.003_0.1_stair_a3/model-12
[0204 06:52:30] Restoring model done.


In [18]:
model.logger.info("Model preparing test")
def custom_apply_tf_op(inputs, output_gate):
    return apply_tf_op(inputs=inputs, session=model.sess, input_gate=model.anc_img, output_gate=output_gate, batch_size=model.args.nbatch//2, dim=4, train_gate=model.istrain)
model.test_k_hash = custom_apply_tf_op(inputs=model.test_image, output_gate=model.anc_embed_k_hash)
model.val_k_hash = custom_apply_tf_op(inputs=model.val_image, output_gate=model.anc_embed_k_hash)

[0204 06:54:54] Model preparing test
batch: 100%|##########| 782/782 [00:05<00:00, 138.65it/s]


In [20]:
from np_op import activate_k_2D

In [22]:
activate_k = 1

In [23]:
model.regen_session()
test_k_activate = activate_k_2D(model.test_k_hash, k=activate_k, session=model.sess) # [ntest, args.d]
val_k_activate = activate_k_2D(model.val_k_hash, k=activate_k, session=model.sess)

batch: 100%|##########| 500/500 [00:00<00:00, 895.24it/s]


In [24]:
model.val_p_dist.shape

(50000, 50000)

In [25]:
model.regen_session()
model.te_te_distance = pairwise_distance_euclid_efficient(input1=model.test_embed, input2=model.test_embed, session=model.sess, batch_size=128)
model.logger.info("Calculating pairwise distance from test embeddings")

batch: 100%|##########| 391/391 [00:01<00:00, 214.67it/s]
[0204 06:56:17] Calculating pairwise distance from test embeddings


In [27]:
model.te_te_distance.shape

(50000, 50000)

In [28]:
k_set =[1,4,16]

In [29]:
model.ncls_val

1000

In [30]:
model.ncls_test

1000

In [32]:
from evaluation import evaluate_hashtable2

In [33]:
performance = evaluate_hashtable2(test_hash_key=test_k_activate, te_te_distance=model.te_te_distance,\
                                  te_te_query_key=test_k_activate, te_te_query_value=model.test_k_hash,\
                                  test_label=model.test_label, ncls_test=model.ncls_test,\
                                  activate_k=activate_k, k_set=k_set, logger=model.logger)

[0204 06:59:37] test_nmi = 0.374
[0204 06:59:37] te_te_suf 1/4/16 = 31.065240447/31.065240447/31.065240447
[0204 06:59:37] te_te_precision_at_k 1/4/16 = .12714/.1098/.088675
[0204 06:59:37] te_te_recall_at_k 1/4/16 = .12714/.27084/.47048
[0204 06:59:37] te_te_precision_ir = 0.012
[0204 06:59:37] te_te_recall_ir = 0.375


In [ ]:
performance_hash = model.test_hash_metric(args.k, K_SET)